In [ ]:
import cv2
import sys, os, datetime
import numpy as np
import pandas as pd
import re
import time
import matplotlib.pyplot as plt

now = datetime.datetime.now()

video_path = "img/bright img.avi"
path = "img"


img = cv2.imread("img/bright img.jpg", cv2.IMREAD_GRAYSCALE)  # queryiamge
plt.imshow(img, cmap='gray')
plt.show()
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("読み込み失敗")
    sys.exit()

# Features
sift = cv2.SIFT.create()
kp_image, desc_image = sift.detectAndCompute(img, None)
# Feature matching
index_params = dict(algorithm=0, trees=5)
search_params = dict()
flann = cv2.FlannBasedMatcher(index_params, search_params)


filename = path+"/video_LocalOptical.avi"
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
fps = 115
# size = (480, 640)
# writer = cv2.VideoWriter(filename, fourcc, fps, size)
flag = True

for i in range(100):
    ret, frame = cap.read()
    

    if flag==True:
        writer = cv2.VideoWriter(filename, fourcc, fps, (frame.shape[1], frame.shape[0]))
        time.sleep(1)
        flag = False


    if ret:
        # frame = cv2.resize(frame, (640, 480))
        grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # trainimage
        kp_grayframe, desc_grayframe = sift.detectAndCompute(grayframe, None)
        matches = flann.knnMatch(desc_image, desc_grayframe, k=2)
        good_points = []
        for m, n in matches:
            if m.distance < 0.6 * n.distance:
                good_points.append(m)
        # img3 = cv2.drawMatches(img, kp_image, grayframe, kp_grayframe, good_points, grayframe)
        # Homography
        if len(good_points) > 10:
            query_pts = np.float32([kp_image[m.queryIdx].pt for m in good_points]).reshape(-1, 1, 2)
            train_pts = np.float32([kp_grayframe[m.trainIdx].pt for m in good_points]).reshape(-1, 1, 2)
            matrix, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0)
            matches_mask = mask.ravel().tolist()
            # Perspective transform
            h, w = img.shape
            pts = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1, 1, 2)
            dst = cv2.perspectiveTransform(pts, matrix)
            homography = cv2.polylines(frame, [np.int32(dst)], True, (255, 0, 0), 3)
            plt.imshow(homography, cmap='gray')
            # writer.write(homography)
            # writer.write(grayframe)
        else:
            plt.imshow(grayframe, cmap='gray')
            # writer.write(grayframe)
        
        # writer.write(grayframe)
        # plt.imshow(grayframe, cmap='gray')
        plt.show()

    else:
        break

cap.release()
writer.release()

print("Finished")


In [ ]:
import cv2
import numpy as np

# テンプレート画像の読み込み
template = cv2.imread('img/bright img.jpg', cv2.IMREAD_GRAYSCALE)

# 動画の読み込み
cap = cv2.VideoCapture('img/bright img.avi')
if not cap.isOpened():
    print("読み込み失敗")
    sys.exit()

# テンプレートマッチングの手法を指定
method = cv2.TM_CCOEFF_NORMED

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # trainimage
    print(frame.shape)
    if not ret:
        break
    
    # テンプレートマッチング
    result = cv2.matchTemplate(frame, template, method)

    # マッチング結果から類似度が最大の位置を求める
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

    # テンプレートの幅と高さを取得
    w, h = template.shape[1], template.shape[0]

    # 類似度が最大の位置に矩形を描画
    top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)

    # 結果を表示
    plt.imshow(frame, cmap='gray')
    plt.show()
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

cap.release()
cv2.destroyAllWindows()


: 

In [ ]:
import cv2
import numpy as np

# テンプレート画像の初期読み込み
template = cv2.imread('img/bright img.jpg', cv2.IMREAD_GRAYSCALE)

# 動画の読み込み
cap = cv2.VideoCapture('img/bright img.avi')
if not cap.isOpened():
    print("読み込み失敗")
    sys.exit()

# テンプレートマッチングの手法を指定
method = cv2.TM_CCOEFF_NORMED

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # trainimage
    if not ret:
        break

    # テンプレートマッチング
    result = cv2.matchTemplate(frame, template, method)

    # マッチング結果から類似度が最大の位置を取得
    _, max_val, _, max_loc = cv2.minMaxLoc(result)

    # テンプレートの幅と高さを取得
    w, h = template.shape[::-1]

    # 類似度が最大の位置に矩形を描画
    top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)

    # マッチングした領域を新たなテンプレートとして保存
    new_template = frame[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
    # cv2.imwrite('img/new_template.jpg', new_template)

    # 新しいテンプレートを次のフレームで使用
    template = new_template

    # 結果を表示
    plt.imshow(frame, cmap='gray')
    plt.show()

cap.release()
# cv2.destroyAllWindows()


: 